In [13]:
import sys
from pathlib import Path

here = Path.cwd().parent.parent
sys.path.append(str(here))

from lion_core.libs import flatten
from typing import Dict, Any
import json


In [14]:
a = {"a": [1, [[2]], [{3}]], "g":{"b": [4]}}


a1 = flatten(a)
print(a1)

{'a|0': 1, 'a|2|0': {3}, 'g|b|0': 4, 'a|1|0|0': 2}


In [17]:
a2 = flatten(a, dynamic=True, coerce_sequence="dict", coerce_keys=False)
a3 = flatten(a, dynamic=True, coerce_sequence="list", coerce_keys=False)
print(a2)
print(a3)

{('a', '0'): 1, ('a', '2', '0'): {3}, ('g', 'b', '0'): 4, ('a', '1', '0', '0'): 2}
{('a', 0): 1, ('a', 2, 0): {3}, ('g', 'b', 0): 4, ('a', 1, 0, 0): 2}


In [16]:
a1 = flatten(a, dynamic=False, coerce_keys=True)
a2 = flatten(a, dynamic=False, coerce_keys=False)
print(a1)
print(a2)

{'a': [1, [[2]], [{3}]], 'g|b': [4]}
{('a',): [1, [[2]], [{3}]], ('g', 'b'): [4]}


In [6]:
a1 = flatten(a, dynamic=False, coerce_sequence=False, coerce_keys=True)
a2 = flatten(a, dynamic=False, coerce_sequence=False, coerce_keys=False)
print(a1)
print(a2)

{'a': [1, [[2]], [{3}]], 'g|b': [4]}
{('a',): [1, [[2]], [{3}]], ('g', 'b'): [4]}
